In [1]:
!pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.2/246.2 kB 602.4 kB/s eta 0:00:00


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
from tqdm import tqdm

In [3]:
SPOTIPY_CLIENT_ID="cbce170827e9428380affa66cf57c720"
SPOTIPY_CLIENT_SECRET="75a9172cda034ef98ee818ca8d8e043b"

In [4]:
songs_uris = pd.read_csv("../input/spotifyuniquesongs/unique_songs.csv", index_col=False)
print(songs_uris.head())

                             track_uris
0  spotify:track:0mojspJSs87KOsjvE2KuJF
1  spotify:track:199PQ256i0Ztnrdqx2wMvA
2  spotify:track:6GEIp2UvaiAlUT5cujmUPL
3  spotify:track:1eLJfeb0gqOIUkb22qI549
4  spotify:track:5t5NWLijqE7LfuXvhAgdKo


In [5]:
songs_uris = songs_uris.track_uris.to_list()

In [6]:
client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [7]:
features_keys = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type']

In [8]:
songs_uris = [songs_uris[i:i+100] for i in range(0, len(songs_uris), 100)]
print(songs_uris[0])

['spotify:track:0mojspJSs87KOsjvE2KuJF', 'spotify:track:199PQ256i0Ztnrdqx2wMvA', 'spotify:track:6GEIp2UvaiAlUT5cujmUPL', 'spotify:track:1eLJfeb0gqOIUkb22qI549', 'spotify:track:5t5NWLijqE7LfuXvhAgdKo', 'spotify:track:4kqsUYcqAgwMc3yqpyj9GY', 'spotify:track:3pmKzeNzD5fILLU0OzZtmH', 'spotify:track:6pDZfypNNSfset3fmpDWHi', 'spotify:track:3Je8RHcdTJ8NGG3krmCHUd', 'spotify:track:16dQ3mPiyuTMr1RnaIGFhp', 'spotify:track:4Du44wbQsEtFQM7HKfvwZa', 'spotify:track:1U0zKU9xSYnYS6DJfzyOsr', 'spotify:track:7BXqIH9zp3TCz23PNhz87l', 'spotify:track:2oUZiX3Lb6XFznDpNZTQ9r', 'spotify:track:0cPKTYnezPWLvz8TxT30RT', 'spotify:track:230YFIJp4PXLDQyvIWlcaq', 'spotify:track:7wofq9SjgI1bvPWQ80rLtF', 'spotify:track:0nTgrhgenyQaBPfzsXwUOo', 'spotify:track:4AvxjIh93FEL0XxS4SlLnl', 'spotify:track:2VASWjU01graj5236bX6ol', 'spotify:track:20MllvmYvXEMj2PQvm3IKf', 'spotify:track:3EAW9dEhzNGOjxX6l7txsV', 'spotify:track:5wYG7LnAI1pQU6kwFaJswW', 'spotify:track:1SZKBlRe0uTWCWRUlEXI4D', 'spotify:track:0UhWvxXuYBbYa2pvDU2fqk',

In [9]:
songs_features = {}

for track_list in tqdm(songs_uris):
    try:
        f_list = sp.audio_features(track_list)
        f_list = [{k: d[k] for k in d if k in features_keys} for d in f_list]
        for track_uri, track_features in zip(track_list, f_list):
            songs_features[track_uri] = track_features
    except:
        continue

print(len(songs_features))

100%|██████████| 22623/22623 [53:25<00:00,  7.06it/s]

2252192


In [10]:
songs_df = [[track_uri, ] +  list(v.values()) for track_uri, v in songs_features.items()]

In [11]:
print(songs_df[0:2])

[['spotify:track:0mojspJSs87KOsjvE2KuJF', 0.723, 0.814, 7, -4.977, 1, 0.151, 0.0335, 0, 0.101, 0.678, 172.153, 'audio_features'], ['spotify:track:199PQ256i0Ztnrdqx2wMvA', 0.438, 0.259, 7, -15.196, 1, 0.0293, 0.888, 0.0669, 0.0805, 0.68, 83.483, 'audio_features']]


In [12]:
songs_df = pd.DataFrame(songs_df, columns=["track_uri"]+features_keys)

In [13]:
songs_df.head()

,track_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type
0,spotify:track:0mojspJSs87KOsjvE2KuJF,0.723,0.814,7,-4.977,1,0.1510,0.033500,0.000000,0.1010,0.678,172.153,audio_features
1,spotify:track:199PQ256i0Ztnrdqx2wMvA,0.438,0.259,7,-15.196,1,0.0293,0.888000,0.066900,0.0805,0.680,83.483,audio_features
2,spotify:track:6GEIp2UvaiAlUT5cujmUPL,0.399,0.984,5,-3.094,0,0.0667,0.000003,0.609000,0.0440,0.176,75.031,audio_features
3,spotify:track:1eLJfeb0gqOIUkb22qI549,0.223,0.716,2,-8.521,1,0.0348,0.001160,0.235000,0.1550,0.739,176.756,audio_features
4,spotify:track:5t5NWLijqE7LfuXvhAgdKo,0.788,0.810,8,-6.155,1,0.0455,0.482000,0.000268,0.0504,0.941,101.655,audio_features


In [14]:
songs_df.to_csv("songs_features.csv", index=False)

# 